In [7]:
# Install required packages (standard library modules like datetime do not require pip)
!pip install requests pandas numpy plotly


DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/acnportal-0.3.3-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================

# Load yearly data (replace paths with actual ones)
df_2018 = pd.read_csv('YEAR/sessions_filtered_2018.csv')
df_2019 = pd.read_csv('YEAR/sessions_filtered_2019.csv')
df_2020 = pd.read_csv('YEAR/sessions_filtered_2020.csv')
df_2021 = pd.read_csv('YEAR/sessions_filtered_2021.csv')


# =============================================================================
# 2. PROCESSING FUNCTION FOR EACH YEARLY DATAFRAME
# =============================================================================

def process_sessions_df(df_raw, year_label):
    """Cleans and processes a sessions DataFrame for one year."""
    df = df_raw.dropna(subset=[
        'sessionID', 'connectionTime', 'disconnectTime',
        'doneChargingTime', 'kWhDelivered'
    ]).copy()

    # Convert timestamps
    for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

    # Compute session duration (in hours)
    df['session_duration'] = (
        df['disconnectTime'] - df['connectionTime']
    ).dt.total_seconds() / 3600.0

    # Filter sessions between 0 and 12 hours
    df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

    # Normalize hour: shift so 8:00 becomes 0 (local time = UTC-8)
    df['hour'] = (df['connectionTime'].dt.hour - 8) % 24

    # Compute charging duration and filter out invalid entries
    df['charging_duration'] = (
        df['doneChargingTime'] - df['connectionTime']
    ).dt.total_seconds() / 3600.0
    df = df[df['charging_duration'] > 0]

    # Compute average power output
    df['power_output'] = df['kWhDelivered'] / df['charging_duration']

    # Tag with year
    df['year'] = year_label

    return df


# Process all years
df_2018_clean = process_sessions_df(df_2018, 2018)
df_2019_clean = process_sessions_df(df_2019, 2019)
df_2020_clean = process_sessions_df(df_2020, 2020)
df_2021_clean = process_sessions_df(df_2021, 2021)

df_all = pd.concat(
    [df_2018_clean, df_2019_clean, df_2020_clean, df_2021_clean],
    ignore_index=True
)


# =============================================================================
# 3. ARRIVAL RATE BY HOUR
# =============================================================================

arrival_rate = df_all.groupby(['year', 'hour']).size().reset_index(name='arrival_rate')

fig_arrival = px.line(
    arrival_rate,
    x='hour',
    y='arrival_rate',
    color='year',
    markers=True,
    labels={'hour': 'Hour of Day', 'arrival_rate': 'Number of Sessions'},
    title='Session Arrival Rate by Hour (All Years)'
)
fig_arrival.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig_arrival.show()


# =============================================================================
# 4. RATIO OF SESSION DURATIONS BY HOUR
# =============================================================================

# Sum of durations by year and hour
sum_durations = df_all.groupby(['year', 'hour'])['session_duration'].sum().reset_index(name='sum_duration')

# Normalize to compute ratio per year
def compute_ratio(group):
    total = group['sum_duration'].sum()
    group['duration_ratio'] = group['sum_duration'] / total
    return group

sum_durations = sum_durations.groupby('year').apply(compute_ratio)

fig_ratio = px.line(
    sum_durations,
    x='hour',
    y='duration_ratio',
    color='year',
    markers=True,
    labels={'hour': 'Hour of Day', 'duration_ratio': 'Fraction of Total Session-Hours'},
    title='Ratio of Session Durations by Hour (All Years)'
)
fig_ratio.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig_ratio.show()


# =============================================================================
# 5. DAILY AVERAGE POWER CALCULATION PER YEAR
# =============================================================================

def compute_daily_avg_power(df_clean, year_label):
    """Computes daily average power output from session start/end events."""
    df_start = df_clean[['connectionTime', 'power_output']].copy()
    df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

    df_end = df_clean[['doneChargingTime', 'power_output']].copy()
    df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
    df_end['power_output'] = -df_end['power_output']

    events_df = pd.concat([df_start, df_end]).sort_values('timestamp')
    events_df['total_power'] = events_df['power_output'].cumsum()
    events_df.set_index('timestamp', inplace=True)

    daily_avg = (
        events_df['total_power']
        .resample('D')
        .mean()
        .fillna(0)
        .reset_index()
    )
    daily_avg['year'] = year_label
    return daily_avg


# Compute daily average power for all years
daily_avg_2018 = compute_daily_avg_power(df_2018_clean, 2018)
daily_avg_2019 = compute_daily_avg_power(df_2019_clean, 2019)
daily_avg_2020 = compute_daily_avg_power(df_2020_clean, 2020)
daily_avg_2021 = compute_daily_avg_power(df_2021_clean, 2021)

daily_all = pd.concat(
    [daily_avg_2018, daily_avg_2019, daily_avg_2020, daily_avg_2021],
    ignore_index=True
)

fig_demand = px.line(
    daily_all,
    x='timestamp',
    y='total_power',
    color='year',
    labels={'timestamp': 'Date', 'total_power': 'Average System Demand (kW)'},
    title='Average kW Demand of the Garage Throughout Each Year'
)
fig_demand.show()


# =============================================================================
# 6. INSIGHTS (OPTIONAL PRINTS)
# =============================================================================

print("Arrival Rate by Hour (first few rows):")
print(arrival_rate.head())

print("\nRatio of Total Session-Hours by Hour (first few rows):")
print(sum_durations.head())

print("\nDaily Average System Demand (first few rows):")
print(daily_all.head())


/var/folders/v0/btbypqg16m19ftg41s8lglrm0000gn/T/ipykernel_62968/3109234243.py:104: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Arrival Rate by Hour (first few rows):
   year  hour  arrival_rate
0  2018     0            58
1  2018     1            21
2  2018     2            23
3  2018     3           136
4  2018     4            73

Ratio of Total Session-Hours by Hour (first few rows):
        year  hour  sum_duration  duration_ratio
year                                            
2018 0  2018     0    228.405278        0.003181
     1  2018     1     72.245000        0.001006
     2  2018     2     77.466389        0.001079
     3  2018     3    942.560000        0.013126
     4  2018     4    261.582222        0.003643

Daily Average System Demand (first few rows):
   timestamp  total_power  year
0 2018-04-25    34.177346  2018
1 2018-04-26    30.149907  2018
2 2018-04-27    30.727468  2018
3 2018-04-28    18.024874  2018
4 2018-04-29    19.785868  2018


In [9]:
# =============================================================================
# 7. BAR CHART: TOTAL kWh DELIVERED PER YEAR
# =============================================================================

# Calculate total kWh delivered per year
yearly_kwh = df_all.groupby('year')['kWhDelivered'].sum().reset_index(name='total_kwh')

# Convert 'year' to string for categorical x-axis in bar chart
yearly_kwh['year'] = yearly_kwh['year'].astype(str)

# Create bar chart with labels rounded to 2 decimal places
fig_kwh = px.bar(
    yearly_kwh,
    x='year',
    y='total_kwh',
    text=yearly_kwh['total_kwh'].round(2),
    labels={
        'year': 'Year',
        'total_kwh': 'Total kWh Delivered'
    },
    title='Total kWh Delivered per Year'
)

# Improve layout readability
fig_kwh.update_layout(
    xaxis=dict(dtick=1),
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig_kwh.show()


In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

# =============================================================================
# 1. LOAD THE DATA
# =============================================================================
df = pd.read_csv('YEAR/sessions_filtered_2018.csv')

# =============================================================================
# 2. DATA CLEANING AND FILTERING
# =============================================================================
df.dropna(subset=['sessionID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered'], inplace=True)

# Convert to datetime
for col in ['connectionTime', 'disconnectTime', 'doneChargingTime']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Drop rows with failed datetime conversion
df.dropna(subset=['connectionTime', 'disconnectTime'], inplace=True)

# Calculate session duration in hours (using disconnectTime)
df['session_duration'] = (df['disconnectTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive and extreme durations
df = df[(df['session_duration'] > 0) & (df['session_duration'] < 12)]

# =============================================================================
# Normalize hour: shift such that 8:00 becomes 0 (the garage is in Los Angeles; times are in GMT)
# =============================================================================
df['hour'] = (df['connectionTime'].dt.hour - 8) % 24

# =============================================================================
# PLOTTING ARRIVAL RATE AND AVERAGE SESSION DURATION BY HOUR
# =============================================================================
# Compute arrival rate and average session duration
arrival_rate = df.groupby('hour').size()
avg_duration = df.groupby('hour')['session_duration'].mean()

# Convert to DataFrame for Plotly
metrics_df = pd.DataFrame({
    'hour': arrival_rate.index,
    'arrival_rate': arrival_rate.values,
    'avg_duration': avg_duration.values
})

# Plot arrival rate
fig1 = px.line(metrics_df, x='hour', y='arrival_rate', markers=True,
               labels={'hour': 'Hour of Day', 'arrival_rate': 'Number of Sessions'},
               title='Session Arrival Rate by Hour')
fig1.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

# Plot average session duration
fig2 = px.line(metrics_df, x='hour', y='avg_duration', markers=True,
               labels={'hour': 'Hour of Day', 'avg_duration': 'Average Duration (hours)'},
               title='Average Session Duration by Hour')
fig2.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))

fig1.show()
fig2.show()

print("Arrival Rate by Hour:")
print(arrival_rate.sort_index())
print("\nAverage Session Duration by Hour (in hours):")
print(avg_duration.sort_index())

# =============================================================================
# COMPUTE CHARGING DURATION AND POWER OUTPUT
# =============================================================================
# Calculate charging duration in hours (time during which the car charged)
df['charging_duration'] = (df['doneChargingTime'] - df['connectionTime']).dt.total_seconds() / 3600.0

# Filter out non-positive charging durations
df = df[df['charging_duration'] > 0]

# Calculate average power output in kilowatts (kW)
df['power_output'] = df['kWhDelivered'] / df['charging_duration']

# =============================================================================
# COMPUTE SYSTEM DEMAND OVER TIME
# =============================================================================
# For each session, assume constant power output during charging.
# Create event records: at connectionTime, add the session's power_output; at doneChargingTime, subtract it.
df_start = df[['connectionTime', 'power_output']].copy()
df_start.rename(columns={'connectionTime': 'timestamp'}, inplace=True)

df_end = df[['doneChargingTime', 'power_output']].copy()
df_end.rename(columns={'doneChargingTime': 'timestamp'}, inplace=True)
df_end['power_output'] = -df_end['power_output']

# Combine start and end events, then sort by time
events_df = pd.concat([df_start, df_end]).sort_values('timestamp')

# Compute the cumulative sum to derive the instantaneous total power demand
events_df['total_power'] = events_df['power_output'].cumsum()

# Set the timestamp as index for resampling
events_df = events_df.set_index('timestamp')

# Resample to daily frequency to obtain the average system demand per day throughout the year
daily_avg = events_df['total_power'].resample('D').mean().reset_index()

# =============================================================================
# PLOTTING AVERAGE SYSTEM DEMAND OVER THE YEAR
# =============================================================================
fig_demand = px.line(daily_avg,
                     x='timestamp',
                     y='total_power',
                     labels={'timestamp': 'Date', 'total_power': 'Average System Demand (kW)'},
                     title='Average kW Demand of the Garage Throughout the Year')
fig_demand.show()

Arrival Rate by Hour:
hour
0       58
1       21
2       23
3      136
4       73
5      374
6     1051
7     2509
8     2052
9     1026
10     616
11     694
12     716
13     533
14     533
15     606
16     685
17     809
18     650
19     462
20     299
21     226
22     120
23      79
dtype: int64

Average Session Duration by Hour (in hours):
hour
0     3.938022
1     3.440238
2     3.368104
3     6.930588
4     3.583318
5     6.104381
6     7.219160
7     7.204178
8     6.758763
9     5.928131
10    4.581511
11    4.130095
12    3.735627
13    3.026117
14    2.472364
15    2.329036
16    2.326525
17    2.275202
18    2.876412
19    3.322819
20    3.777589
21    3.842763
22    4.287176
23    4.557711
Name: session_duration, dtype: float64


In [12]:
# -----------------------------------------------------------------------------
# 7. Plot average charging duration by hour
# -----------------------------------------------------------------------------

# Compute average charging duration per hour (using doneChargingTime)
charging_avg_duration = df.groupby('hour')['charging_duration'].mean()

# Create a DataFrame for Plotly
charging_metrics_df = pd.DataFrame({
    'hour': charging_avg_duration.index,
    'avg_charging_duration': charging_avg_duration.values
})

# Plot average charging duration by hour
fig3 = px.line(charging_metrics_df, x='hour', y='avg_charging_duration', markers=True,
               labels={'hour': 'Hour of Day', 'avg_charging_duration': 'Average Charging Duration (hours)'},
               title='Average Charging Duration by Hour')
fig3.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig3.show()

print("\nAverage Charging Duration by Hour (in hours):")
print(charging_avg_duration.sort_index())



Average Charging Duration by Hour (in hours):
hour
0     2.730043
1     2.682103
2     2.232089
3     3.724112
4     2.216648
5     3.084427
6     4.075458
7     4.031028
8     3.819704
9     3.559760
10    2.951424
11    2.556119
12    2.209100
13    2.050519
14    1.759493
15    1.847632
16    1.914415
17    1.867143
18    1.863716
19    2.062166
20    2.239421
21    2.501111
22    2.566016
23    2.482686
Name: charging_duration, dtype: float64


In [13]:
# =============================================================================
# FILTER OUT WEEKENDS AND PLOT
# =============================================================================
# Add a weekday column (0 = Monday, 6 = Sunday)
daily_avg['weekday'] = daily_avg['timestamp'].dt.weekday

# Filter: only keep weekdays (0–4)
weekday_avg = daily_avg[daily_avg['weekday'] < 5]

# Plot average system demand excluding weekends
fig_weekday = px.line(weekday_avg,
                      x='timestamp',
                      y='total_power',
                      labels={'timestamp': 'Date', 'total_power': 'Avg Weekday Demand (kW)'},
                      title='Average Weekday kW Demand (Weekends Removed)')
fig_weekday.show()

# =============================================================================
# WEEKLY AVERAGE DEMAND
# =============================================================================
weekly_avg = events_df['total_power'].resample('W').mean().reset_index()

fig_weekly = px.line(weekly_avg,
                     x='timestamp',
                     y='total_power',
                     labels={'timestamp': 'Week', 'total_power': 'Avg Weekly Demand (kW)'},
                     title='Average Weekly kW Demand Over the Year')
fig_weekly.show()


In [14]:
import plotly.express as px

# Compute the ratio between charging duration and session duration
df['charging_ratio'] = df['charging_duration'] / df['session_duration']

# Group data by the shifted 'hour' and calculate the average ratio per hour
ratio_by_hour = df.groupby('hour')['charging_ratio'].mean().reset_index()

# Create a line plot to visualize the average ratio by hour
fig_ratio = px.line(ratio_by_hour, x='hour', y='charging_ratio', markers=True,
                    labels={'hour': 'Hour of Day', 
                            'charging_ratio': 'Average Ratio (Charging Duration / Session Duration)'},
                    title='Average Ratio of Charging Duration to Session Duration by Hour')
fig_ratio.update_layout(xaxis=dict(tickmode='linear', tick0=0, dtick=1))
fig_ratio.show()

print("\nAverage Ratio of Charging Duration to Session Duration by Hour:")
print(ratio_by_hour.sort_values('hour'))


Average Ratio of Charging Duration to Session Duration by Hour:
    hour  charging_ratio
0      0        0.745822
1      1        0.828367
2      2        0.714000
3      3        0.628274
4      4        0.815523
5      5        0.646803
6      6        0.625025
7      7        0.614039
8      8        0.631886
9      9        0.681665
10    10        0.736975
11    11        0.697498
12    12        0.687392
13    13        0.759258
14    14        0.813690
15    15        0.853816
16    16        0.861581
17    17        0.868212
18    18        0.828434
19    19        0.825869
20    20        0.792517
21    21        0.824914
22    22        0.733174
23    23        0.587255


In [15]:
station_counts = df['stationID'].value_counts().reset_index()
station_counts.columns = ['stationID', 'Usage']

space_counts = df['spaceID'].value_counts().reset_index()
space_counts.columns = ['spaceID', 'Usage']

site_counts = df['siteID'].value_counts().reset_index()
site_counts.columns = ['siteID', 'Usage']

# Create an interactive bar chart for station usage
fig_station = px.bar(
    station_counts,
    x='stationID',
    y='Usage',
    title='Usage by Station',
    labels={'stationID': 'Station ID', 'Usage': 'Number of Sessions'}
)
fig_station.show()

# Create an interactive bar chart for space usage
fig_space = px.bar(
    space_counts,
    x='spaceID',
    y='Usage',
    title='Usage by Space',
    labels={'space': 'Space', 'Usage': 'Number of Sessions'}
)
fig_space.show()

# Create an interactive bar chart for site usage
fig_site = px.bar(
    site_counts,
    x='siteID',
    y='Usage',
    title='Usage by Site',
    labels={'site': 'Site', 'Usage': 'Number of Sessions'}
)
fig_site.show()